In [1]:
import pandas as pd

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [114]:
assumptions = pd.DataFrame({
    'LS' : [82050, 58266, 49723],
    'LT' : [1, 2, 3],
    'int_inventory' : [10000, 10000, 10000]
})
assumptions.index = ["Japan", "Germany", "USA"]

customer_orders = {
    "Japan": [35000] * 24,
    "Germany": [30000] * 24,
    "USA": [23000] * 24
} 

In [113]:
assumptions

,LS,LT,int_inventory
Japan,82050,1,10000
Germany,58266,2,10000
USA,49723,3,10000


In [100]:
customer_orders['Japan']

[35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000,
 35000]

In [101]:
# deviding the demand to weeks
forecast = {
    'Japan': [82102] * 24,   
    'Germany': [110473] * 24,
    'USA': [119107] * 24
}

In [102]:
forecast

{'Japan': [82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102,
  82102],
 'Germany': [110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473,
  110473],
 'USA': [119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107,
  119107]}

In [105]:
weeks = 24

In [106]:
def MPS(country, LS, forecast, customer_orders):
    proj_available_bal = [0] * weeks
    mps = [0] * weeks
    avail_to_promise = [0] * weeks

    # initial inventory
    mps[0] = round(customer_orders[country][0] / LS) * LS
    proj_available_bal[0] = mps[0] - customer_orders[country][0]
    avail_to_promise[0] = proj_available_bal[0] - sum(customer_orders[country][1:3])

    for t in range(1, weeks):
        # First, calculate available balance
        proj_available_bal[t] = proj_available_bal[t - 1] + mps[t - 1] - max(customer_orders[country][t], forecast[country][t])
        
        # If available balance is negative, we need to adjust MPS
        if proj_available_bal[t] < 0:
            mps[t] = round(abs(proj_available_bal[t]) / LS) * LS
            proj_available_bal[t] = mps[t]  # Reset proj_available_bal to the new MPS

    last_mps_index = -1
    for t in range(weeks):
        if mps[t] > 0:
            if last_mps_index != -1:
                avail_to_promise[last_mps_index] = mps[last_mps_index] - sum(customer_orders[country][last_mps_index:t])
            last_mps_index = t

    if last_mps_index != -1:
        avail_to_promise[last_mps_index] = mps[last_mps_index] - sum(customer_orders[country][last_mps_index:])

    df = pd.DataFrame({
        "Forecast": forecast[country],
        "Customer Orders": customer_orders[country],
        "Proj. Avail. Bal.": proj_available_bal,
        "MPS": mps,
        "Avail. To Promise": avail_to_promise,
    }, index=[f"Week {i+1}" for i in range(weeks)])

    return df.T , mps


In [116]:
for country in ['Japan', 'Germany', 'USA']:
    print(f'MPS table for {country}')
    print('----------------------------------------------------------------------------------')
    df, mps = MPS(
        country,
        assumptions.loc[country, 'LS'],
        forecast,  
        customer_orders,
    )
    print(df, '\n')

MPS table for Japan
----------------------------------------------------------------------------------
                   Week 1  Week 2  Week 3  Week 4  Week 5  Week 6  Week 7  \
Forecast            82102   82102   82102   82102   82102   82102   82102   
Customer Orders     35000   35000   35000   35000   35000   35000   35000   
Proj. Avail. Bal.  -35000   82050   81998       0   82050   81998       0   
MPS                     0   82050       0       0   82050       0       0   
Avail. To Promise -105000  -22950       0       0  -22950       0       0   

                   Week 8  Week 9  Week 10  ...  Week 15  Week 16  Week 17  \
Forecast            82102   82102    82102  ...    82102    82102    82102   
Customer Orders     35000   35000    35000  ...    35000    35000    35000   
Proj. Avail. Bal.   82050   81998        0  ...    81998        0    82050   
MPS                 82050       0        0  ...        0        0    82050   
Avail. To Promise  -22950       0        0  